In [ ]:
import os
import pandas as pd
import numpy as np

print(os.getcwd())
PATH = os.getcwd()
PATH = PATH + "/NGSIM.csv" # this is the original csv file that you want to extract contact duration from
print(PATH)

In [ ]:
df = pd.read_csv(PATH, encoding='utf-8', engine='python')

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
df = df.sort_values('Global_Time')

In [ ]:
num_samples = df.shape[0]
num_features = df.shape[1]
print("number of samples = " + str(num_samples))
print("number of features = " + str(num_features))
print(df.columns)

In [ ]:
data = df.to_numpy()

In [ ]:
i = 0
j = 0

Big_dictionary = {} # initialize a dictionary that stores all states of links

for i in range(num_samples):
    
    time_holder = data[i][3]
    links_now = {}
    
    #if i > 2000:  # number of rows we want to process
        #break
    
    small_dictionary = {} # stores the links at this point in time(time_holder)
    
    while data[j][3] == time_holder : # current vehicle 
        next_vehicle = j + 1 # check the vehicles below it
        if next_vehicle >= num_samples:
            break
                    
        while data[next_vehicle][3] == time_holder:
            # get relative location, and convert from feet to meters
            relative_location = [(data[j][6] - data[next_vehicle][6])/3.28,(data[j][7] - data[next_vehicle][7])/ 3.28]
            relative_distance = np.linalg.norm(relative_location)
                       
            if relative_distance <= 100:  # in the range of 50 meters
                
                vehicle_pair = [data[j][1], data[next_vehicle][0]] 
                a = data[j][0:num_features].tolist()
                b = data[next_vehicle][1:num_features].tolist()
                a.extend(b)
                link_features = a                
                vehicle_pair = tuple(vehicle_pair) 
                
                small_dictionary.update({vehicle_pair:link_features})                
                                        
            next_vehicle = next_vehicle + 1
            if next_vehicle >= num_samples:
                break
        
        j = j + 1
        if j % 100000 == 0:
            print("This is the " + str(j) + "th row") 
            # this helps to visualize the progress of processing/how much we've done
          
    if len(small_dictionary.keys()) != 0:
        Big_dictionary.update({time_holder:small_dictionary})
                                
    i = j

print("now we have the big dictionary")

In [ ]:
existing_links = {} # list that stores all the links that exist at time_current

contact_info = [] # list that stores all the data points with contact duration. 

for time_current, linkinfo_current in Big_dictionary.items(): 
    # starting from the second key-value pair, key is time, value is dic of link info
    #iterate through time
                
    e_cpy = existing_links.copy() # to avoid RuntimeError: dictionary changed size during iteration
    

    # iterate through existing links by time, to check if it ends at this time(time_current)
    for time_existing, linkinfo_existing in e_cpy.items():  

        l_e_t = linkinfo_existing.copy()
        # iterate through all the current links at a particular time
        for existing_vehicle_pair, existing_features in l_e_t.items():
            link_break = True 
            
            for current_vehicle_pair, current_features in linkinfo_current.items():
                
                if sorted(existing_vehicle_pair) == sorted(current_vehicle_pair):
                    link_break = False
                    break # know that this existing_vehicle pair is still in contact, move on to check the next
            
            if link_break == True:                   
                # if it doesn't equal to any of the current_vehicle_pair, then we know that the contact ends here
                # compute link_duration
                link_duration = time_current - time_existing 
                
                # store all info(existing_vehicle_pair, existing_features, link_duration) to a new np array                        
                new_data = list(existing_vehicle_pair)
                existing_features.append(link_duration)
                new_data.extend(existing_features)
                #then append this numpy array to the 'final_output' numpy array
                contact_info.append(new_data)

                # a brief view of the data
                #print(existing_vehicle_pair, existing_time, link_duration)
                                
                # remove the link that ends at this time from 'existing_links'
                existing_links.get(time_existing).pop(existing_vehicle_pair)
                leng = len(existing_links.get(time_existing).keys())
                if(leng == 0):
                    existing_links.pop(time_existing)
                
    
    ###############################################################################################################
    #iterate through current links to see if it's a new link.
    
    e_cpy = existing_links.copy() 
    
    for current_vehicle_pair, current_features in linkinfo_current.items(): 
               
        #
        if len(e_cpy.keys()) == 0: # if there's no existing link, then add all current links to existing_links
            existing_links.update( {time_current:linkinfo_current} )
            continue
        
        is_new = True # if it's a new link, set the default value as True
        
        for time_existing, linkinfo_existing in e_cpy.items(): 
            
            l_e_t = linkinfo_existing.copy() 
                        
            for existing_vehicle_pair, existing_features in l_e_t.items():
                if sorted(current_vehicle_pair) == sorted(existing_vehicle_pair):
                    is_new = False
                    break # know that this current_vehicle_pair is already in contact, move on to check the next
                
        # if it doesn't equal to any of the existing_vehicle pair, then we know that the it's a new link 
        # add this to the existing_links            
        if is_new == True:
            new_link = {current_vehicle_pair: current_features}
            existing_links.update( {time_current:new_link} )
            
#convert the list to a numpy array        
output_array = np.asarray(contact_info)    

# convert the numpy array to a csv file.
what_we_want = np.savetxt("ouput_t.csv", output_array, delimiter=",",fmt = "%s")

# after this, we still need to further process the csv file a little bit to get the right format

